# Segment nuclei and quantify ploidy

## Workflow

1. Segment nuclei with StarDist
    - stardist works best when nuclei are nearly isometric, so 
    - upscale images in z by ~4x
    - predict labels with StarDist model
    - downscale predicted labels to original dimensions
2. correct segmentations and quantify ploidy
    - manually correct any bad intestine segmentations in napari
    - with new label images, draw over intestine and pachytene nuclei to identify them
    - quantify image statistics (including intensity) from all labels
3. expand germline labels and re-quantify germline ploidy
4. do further processing in R
    - filter ROIs of interest
    - background subtraction
    - within-worm ploidy normalization
    - plotting
   
*Several other, potentially, useful functions*

- `skimage.segmentation.relabel_sequential` will renumber label images so that the numbers are as small as possible and sequential
- `skimage.morphology.remove_small_objects` can filter out labels below a particular size threshold

## 1. predict labels with StarDist

### a. upscale images using `skimage.transform.resize`

- use bilinear interpolation for images `order = 1`
- use nearest neighbor interpolation for labels `order = 0` 
- when downscaling labels, also turn off antialiasing `anti_aliasing = False`

Notes: Replicate 1 (full_images_V2) need to be upscaled 4X to become isotropic since they used a 0.6 um z-step, but replicate 2 (full_images_V2_R2) only need 2X upscaling since they used a 0.3 um z-step.)

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tifffile import imread, imwrite
from skimage.transform import resize

data_dir = Path('full_images_V2_R2')
orig_images_dir = data_dir/'original_tifs'
upsc_images_dir = data_dir/'upscaled_tifs'
orig_labels_dir = data_dir/'original_labels'
upsc_labels_dir = data_dir/'upscaled_labels'
unup_labels_dir = data_dir/'un_upscaled_labels'
csv_dir = data_dir/'csvs'

image_path_list = [file for file in orig_images_dir.iterdir() if file.suffix == '.tif']
image_path_list.sort()

def concat_csvs(directory):
    # concatenates all csvs in directory. returns concatenated df
    df_list = []
    file_list = [file for file in directory.iterdir() if file.suffix == '.csv']
    file_list.sort()
    
    for file in file_list:
        df = pd.read_csv(file, index_col = 0)
        df_list.append(df)
        
    df_concat = pd.concat(df_list, axis = 0, ignore_index = False).reset_index(drop = True)
    return df_concat

In [2]:
for image_path in image_path_list:

    image_id = image_path.stem
    upsc_im_path = upsc_images_dir/f'{image_id}_upz.tif'
    
    if upsc_im_path.is_file():
        print(f'{image_id} already upscaled')
        continue
    
    print(f'upscaling {image_id}')
#     orig_im_path = orig_images_dir/f'{image_id}.tif'
    orig_im = imread(image_path)

    z,y,x = orig_im.shape 
    scaled_shape = (2*z, y, x) #R1 should be 4*z, R2 should be 2*z
    
    upsc_im = resize(orig_im, scaled_shape, order = 1, preserve_range = True)
    imwrite(upsc_im_path, upsc_im)

upscaling 001-bmin_F1
upscaling 002-bmin_F1
upscaling 003-bplu_F1
upscaling 004-bplu_F1
upscaling 005-bplu_F1
upscaling 006-bplu_F1
upscaling 007-bmin_F1
upscaling 008-bplu_F1
upscaling 009-bplu_F1
upscaling 010-bplu_F1
upscaling 011-bmin_F1
upscaling 012-bmin_F1
upscaling 013-bmin_F1
upscaling 014-bplu_F1
upscaling 015-bmin_F1
upscaling 016-bmin_F1
upscaling 017-bplu_F1
upscaling 101-bmin_F1
upscaling 102-bplu_F1
upscaling 103-bplu_F1
upscaling 104-bmin_F1
upscaling 105-bplu_F1
upscaling 106-bmin_F1
upscaling 107-bmin_F1
upscaling 108-bplu_F1
upscaling 109-bmin_F1
upscaling 110-bplu_F1
upscaling 111-bmin_F1
upscaling 112-bplu_F1
upscaling 113-bplu_F1
upscaling 114-bplu_F1
upscaling 115-bmin_F1
upscaling Capture 10_bminF2_01
upscaling Capture 1_bminF2_01
upscaling Capture 1_bminF2_02
upscaling Capture 1_bminF2_03
upscaling Capture 2_bminF2_01
upscaling Capture 2_bminF2_02
upscaling Capture 2_bminF2_03
upscaling Capture 3_bminF2_01
upscaling Capture 3_bminF2_02
upscaling Capture 3_bminF

### b. predict labels with StarDist

In [2]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import pickle

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist.models import StarDist3D

files = sorted(glob('full_images_V2_R2/upscaled_tifs/*.tif'))
files = [f for f in files if not (upsc_labels_dir/f'{Path(f).stem}_labels.tif').is_file()]    
file_stems = [Path(f).stem for f in files]
print(file_stems)

X = list(map(imread,files))

n_channel = 1 if X[0].ndim == 3 else X[0].shape[-1]
axis_norm = (0,1,2)   # normalize channels independently

model = StarDist3D(None, name='stardist_V2_isotropic', basedir='models_V2')

for i in range(len(X)):
    image_normalized = normalize(X[i], 1,99.8, axis=axis_norm)

#     d,h,w = image_normalized.shape
    n_tiles = model._guess_n_tiles(image_normalized)

    labels, details = model.predict_instances(image_normalized, axes='ZYX', n_tiles=n_tiles)
    
    save_labels_path = upsc_labels_dir/f'{file_stems[i]}_labels.tif'
#     save_labels_path = os.path.join('full_images_V2/upscaled_labels', file_stems[i] + '_labels.tif')
    save_tiff_imagej_compatible(save_labels_path, labels, axes='ZYX')
    print(f"saved labels at: {save_labels_path}")
    
    save_details_path = upsc_labels_dir/f'{file_stems[i]}_details.pkl'
#     save_details_path = os.path.join('full_images_V2/upscaled_labels', file_stems[i] + '_details.pkl')
    with open(save_details_path, 'wb') as f:
        pickle.dump(details, f)

['001-bmin_F1_upz', '002-bmin_F1_upz', '003-bplu_F1_upz', '004-bplu_F1_upz', '005-bplu_F1_upz', '006-bplu_F1_upz', '007-bmin_F1_upz', '008-bplu_F1_upz', '009-bplu_F1_upz', '010-bplu_F1_upz', '011-bmin_F1_upz', '012-bmin_F1_upz', '013-bmin_F1_upz', '014-bplu_F1_upz', '015-bmin_F1_upz', '016-bmin_F1_upz', '017-bplu_F1_upz', '101-bmin_F1_upz', '102-bplu_F1_upz', '103-bplu_F1_upz', '104-bmin_F1_upz', '105-bplu_F1_upz', '106-bmin_F1_upz', '107-bmin_F1_upz', '108-bplu_F1_upz', '109-bmin_F1_upz', '110-bplu_F1_upz', '111-bmin_F1_upz', '112-bplu_F1_upz', '113-bplu_F1_upz', '114-bplu_F1_upz', '115-bmin_F1_upz', 'Capture 10_bminF2_01_upz', 'Capture 1_bminF2_01_upz', 'Capture 1_bminF2_02_upz', 'Capture 1_bminF2_03_upz', 'Capture 2_bminF2_01_upz', 'Capture 2_bminF2_02_upz', 'Capture 2_bminF2_03_upz', 'Capture 3_bminF2_01_upz', 'Capture 3_bminF2_02_upz', 'Capture 3_bminF2_03_upz', 'Capture 4_bminF2_01_upz', 'Capture 4_bminF2_02_upz', 'Capture 4_bminF2_03_upz', 'Capture 5_bminF2_01_upz', 'Capture 5_b

MemoryError: Unable to allocate 3.19 GiB for an array with shape (427819008,) and data type float64

In [6]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import pickle

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist.models import StarDist3D

files = sorted(glob('full_images_V2_R2/upscaled_tifs/*.tif'))
files = [f for f in files if not (upsc_labels_dir/f'{Path(f).stem}_labels.tif').is_file()]    
file_stems = [Path(f).stem for f in files]
print(file_stems)

model = StarDist3D(None, name='stardist_V2_isotropic', basedir='models_V2')

for i in range(len(files)):
    X = imread(files[i])

    n_channel = 1 if X.ndim == 3 else X.shape[-1]
    axis_norm = (0,1,2)   # normalize channels independently

    image_normalized = normalize(X, 1,99.8, axis=axis_norm)

    n_tiles = model._guess_n_tiles(image_normalized)

    labels, details = model.predict_instances(image_normalized, axes='ZYX', n_tiles=n_tiles)
    
    save_labels_path = upsc_labels_dir/f'{file_stems[i]}_labels.tif'
    save_tiff_imagej_compatible(save_labels_path, labels, axes='ZYX')
    print(f"saved labels at: {save_labels_path}")
    
    save_details_path = upsc_labels_dir/f'{file_stems[i]}_details.pkl'
    with open(save_details_path, 'wb') as f:
        pickle.dump(details, f)

['001-bmin_F1_upz', '002-bmin_F1_upz', '003-bplu_F1_upz', '004-bplu_F1_upz', '005-bplu_F1_upz', '006-bplu_F1_upz', '007-bmin_F1_upz', '008-bplu_F1_upz', '009-bplu_F1_upz', '010-bplu_F1_upz', '011-bmin_F1_upz', '012-bmin_F1_upz', '013-bmin_F1_upz', '014-bplu_F1_upz', '015-bmin_F1_upz', '016-bmin_F1_upz', '017-bplu_F1_upz', '101-bmin_F1_upz', '102-bplu_F1_upz', '103-bplu_F1_upz', '104-bmin_F1_upz', '105-bplu_F1_upz', '106-bmin_F1_upz', '107-bmin_F1_upz', '108-bplu_F1_upz', '109-bmin_F1_upz', '110-bplu_F1_upz', '111-bmin_F1_upz', '112-bplu_F1_upz', '113-bplu_F1_upz', '114-bplu_F1_upz', '115-bmin_F1_upz', 'Capture 10_bminF2_01_upz', 'Capture 1_bminF2_01_upz', 'Capture 1_bminF2_02_upz', 'Capture 1_bminF2_03_upz', 'Capture 2_bminF2_01_upz', 'Capture 2_bminF2_02_upz', 'Capture 2_bminF2_03_upz', 'Capture 3_bminF2_01_upz', 'Capture 3_bminF2_02_upz', 'Capture 3_bminF2_03_upz', 'Capture 4_bminF2_01_upz', 'Capture 4_bminF2_02_upz', 'Capture 4_bminF2_03_upz', 'Capture 5_bminF2_01_upz', 'Capture 5_b

100%|████████████████████████████████████████████████████████████████████████████████| 338/338 [02:35<00:00,  2.18it/s]
__init__.py (43): Converting data type from 'int32' to ImageJ-compatible 'int16'.


saved labels at: full_images_V2_R2\upscaled_labels\001-bmin_F1_upz_labels.tif


100%|████████████████████████████████████████████████████████████████████████████████| 338/338 [02:14<00:00,  2.52it/s]


saved labels at: full_images_V2_R2\upscaled_labels\002-bmin_F1_upz_labels.tif


...

saved labels at: full_images_V2_R2\upscaled_labels\Capture 7_bminF2_01_upz_labels.tif


100%|████████████████████████████████████████████████████████████████████████████████| 338/338 [02:21<00:00,  2.38it/s]


saved labels at: full_images_V2_R2\upscaled_labels\Capture 8_bminF2_01_upz_labels.tif


100%|████████████████████████████████████████████████████████████████████████████████| 338/338 [02:21<00:00,  2.39it/s]


saved labels at: full_images_V2_R2\upscaled_labels\Capture 9_bminF2_01_upz_labels.tif


### c. Downscale labels predicted by isotropic StarDist model

In [2]:
from pathlib import Path
label_path_list = [file for file in upsc_labels_dir.iterdir() if file.suffix == '.tif']

for label_path in label_path_list:
          
    image_id = label_path.stem
    unup_lab_path = unup_labels_dir/f'{image_id}.tif'
    
    if unup_lab_path.is_file():
        print(f'{image_id} labels already un-upscaled')
        continue
    
    print(f' downscaling {image_id}')
    
#     upsc_lab_path = upsc_labels_dir/f'{image_id}.tif'
    upsc_lab = imread(label_path)

    z,y,x = upsc_lab.shape 
    scaled_shape = (z/2, y, x)
    
    unup_lab = resize(upsc_lab, scaled_shape, order = 0, anti_aliasing = False)
    imwrite(unup_lab_path, unup_lab)

 downscaling 001-bmin_F1_upz_labels
 downscaling 002-bmin_F1_upz_labels
 downscaling 003-bplu_F1_upz_labels
 downscaling 004-bplu_F1_upz_labels
 downscaling 005-bplu_F1_upz_labels
 downscaling 006-bplu_F1_upz_labels
 downscaling 007-bmin_F1_upz_labels
 downscaling 008-bplu_F1_upz_labels
 downscaling 009-bplu_F1_upz_labels
 downscaling 010-bplu_F1_upz_labels
 downscaling 011-bmin_F1_upz_labels
 downscaling 012-bmin_F1_upz_labels
 downscaling 013-bmin_F1_upz_labels
 downscaling 014-bplu_F1_upz_labels
 downscaling 015-bmin_F1_upz_labels
 downscaling 016-bmin_F1_upz_labels
 downscaling 017-bplu_F1_upz_labels
 downscaling 101-bmin_F1_upz_labels
 downscaling 102-bplu_F1_upz_labels
 downscaling 103-bplu_F1_upz_labels
 downscaling 104-bmin_F1_upz_labels
 downscaling 105-bplu_F1_upz_labels
 downscaling 106-bmin_F1_upz_labels
 downscaling 107-bmin_F1_upz_labels
 downscaling 108-bplu_F1_upz_labels
 downscaling 109-bmin_F1_upz_labels
 downscaling 110-bplu_F1_upz_labels
 downscaling 111-bmin_F1_upz

## 2. correct segmentations and classify nuclei by cell type

- load images and labels in napari
- manually correct any bad segmentations
- measure intensity with `napari_simpleitk_image_processing.label_statistics`
    - check if roi overlaps with border to exclude partial rois
    - need number of pixels and total intensity to do background subtraction
- create new label images to classify cells by cell type
    - intestine labeled with int ring == label (i.e. int1 gets 1...)
    - pachytene labeled with 1
- classify any labels that overlap with classifier image as cell type
    - save these labels as a dictionary
    - convert dictionary into data frame
- save corrected nuclei labels, classifier images, measurements, and rois
- concatenate data from all images and join measurements to rois

In [ ]:
import napari
from copy import copy
from napari_simpleitk_image_processing import label_statistics

def list_overlapping_labels(labels, overlap_labels, i):
    # returns list of unique labels from `labels` that have any overlap with `overlap_labels`
    mask = overlap_labels == i
    masked_labels = labels * mask
    list_of_labels = np.unique(masked_labels)
    return list_of_labels

for image_path in image_path_list:
    
    # get image id and define file paths
    image_id = image_path.stem
    orig_label_path = unup_labels_dir/f'{image_id}_upz_labels.tif'
    corr_label_path = data_dir/'corrected_labels'/f'{image_id}_corrected_labels.tif'
    measure_csv_path = data_dir/'csvs'/'measurements'/f'{image_id}_measurements.csv'
    rois_csv_path = data_dir/'csvs'/'rois'/f'{image_id}_rois.csv'

    
    # skip images without labels or that have already been measured
    if not orig_label_path.is_file():
        print(f'{image_id} does not have labels. skipping')
        continue
    
    if measure_csv_path.is_file():
        print(f'{image_id} already measured. skipping')
        continue
    
    print(f' processing {image_id}')
    
    # read image and labels
    im = imread(image_path)
    init_labels = imread(orig_label_path)
    out_labels = copy(init_labels)
    
    # initialize class labels
    classes_dict = {}
    for tissue in ['intestine', 'germline']:
        classes_dict[tissue] = np.zeros(shape = out_labels.shape, dtype = 'uint8')
    
    # view images in napari
    viewer = napari.Viewer()
    viewer.add_image(im, contrast_limits = [0, 0.7 * int(np.max(im))])
    labs = viewer.add_labels(out_labels, opacity = 0.5)
    for key, class_image in classes_dict.items():
        viewer.add_labels(class_image, name = key)
        # label int nuclei with number according to int ring
        # labels tetraploid pachytene nuclei with 1
    
    # functions to make editing labels easier
    @viewer.bind_key('z')
    def decrease_z_position(viewer):
        current_z_slice = viewer.dims.point[0]
        viewer.dims.set_point(axis = 0, value = current_z_slice - 1)
    
    @viewer.bind_key('x')
    def increase_z_position(viewer):
        current_z_slice = viewer.dims.point[0]
        viewer.dims.set_point(axis = 0, value = current_z_slice + 1)
        
    @viewer.bind_key('e')
    def erase_label_3D(viewer):
        labs.n_edit_dimensions = 3
        labs.mode = 'fill'
        labs.selected_label = 0
    
    @viewer.bind_key('d') 
    def draw_label_bounds(viewer):
        labs.n_edit_dimensions = 2
        labs.mode = 'paint'
        labs.brush_size = 7
    
    # prevent loop from advancing until viewer is closed
    viewer.show(block=True) 
    
    # save corrected labels and measurements
    imwrite(corr_label_path, out_labels)
    measure_df = pd.DataFrame(label_statistics(im, out_labels, position=True))
    measure_df['image'] = image_id
    measure_df.to_csv(measure_csv_path)
    
    # create dictionary of labels that are in either intestine or pachytene
    roi_dict = {}
    for tissue, class_image in classes_dict.items():     
        # save class labels
        file_name = f'{image_id}_{tissue}.tif'
        file_path = data_dir/'class_labels'/file_name
        imwrite(file_path, class_image)
        
        # list nucleus labels that overlap with class label
        valid_classes = [c for c in np.unique(class_image) if c>0]
        for c in valid_classes:
            list_of_labels = list_overlapping_labels(out_labels, class_image, c)
            roi_dict[(tissue, c)] = list_of_labels
    
    # convert dictionary to df and save as csv
    lab_df = pd.Series(roi_dict).rename_axis(['tissue', 'sub_tissue']).reset_index(name='label').explode('label')
    lab_df['image'] = image_id
    lab_df.to_csv(rois_csv_path)


240129_balminF2_079 already measured. skipping
240129_balminF2_080 already measured. skipping
240129_balminF2_082 already measured. skipping
240129_balminF2_083 already measured. skipping
240129_balminF2_084 already measured. skipping
240129_balminF2_085 already measured. skipping
240129_balmin_009 already measured. skipping
240129_balmin_018 already measured. skipping
240129_balmin_019 already measured. skipping
240129_balmin_022 already measured. skipping
240129_balmin_041 already measured. skipping
240129_balmin_047 already measured. skipping
240129_balmin_053 already measured. skipping
 processing 240129_balmin_055
Assistant skips harvesting pyclesperanto as it's not installed.
 processing 240129_balmin_059
 processing 240129_balmin_063
 processing 240129_balmin_064
 processing 240129_balmin_065
 processing 240129_balmin_066
240129_balmin_123 already measured. skipping
240129_balmin_223 already measured. skipping
240129_balplus_004 already measured. skipping
240129_balplus_011 alre

In [ ]:
### concatenate rois and measurements from all images
measurements_df = concat_csvs(csv_dir/'measurements')
rois_df = concat_csvs(csv_dir/'rois')
merged_df = rois_df.merge(measurements_df, on = ['label', 'image'])

measurements_df.to_csv(csv_dir/'combined_measurements.csv', index = False)
rois_df.to_csv(csv_dir/'combined_rois.csv', index = False)
merged_df.to_csv(csv_dir/'Exp042_raw_ploidy.csv', index = False)

## 3. Expand germline labels and re-quantify ploidy
Raw ploidy measurements from **2** estimate fractional ploidies slightly above expected 2^n values. One explanation is that I'm not fully capturing the signal from each germline nucleus, leading to too low of a normalization value. This can be solved by expanding the germline labels by 1 or 2 pixels to capture all signal.

In [5]:
from napari_simpleitk_image_processing import label_statistics
from skimage.segmentation import expand_labels

for image_path in image_path_list:

    # get image id and define file paths
    image_id = image_path.stem
    corr_label_path = data_dir/'corrected_labels'/f'{image_id}_corrected_labels.tif'
    exp_label_path = data_dir/'expanded2_corrected_labels'/f'{image_id}_expanded_corrected_labels.tif'
    exp_measure_csv_path = data_dir/'csvs'/'expanded2_measurements'/f'{image_id}_expanded_measurements.csv'

    # skip images without labels or that already have expanded labels
    if not corr_label_path.is_file():
        print(f'{image_id} does not have corrected labels. skipping')
        continue
    
    if exp_label_path.is_file():
        print(f'{image_id} already expanded and measured. skipping')
        continue
    
    print(f' processing {image_id}')
    
    # read image and labels
    im = imread(image_path)
    corr_labels = imread(corr_label_path)
    exp_labels = expand_labels(corr_labels, distance = 2)
    
    # save expanded labels and measurements
    imwrite(exp_label_path, exp_labels)
    measure_df = pd.DataFrame(label_statistics(im, exp_labels, position=True))
    measure_df['image'] = image_id
    measure_df.to_csv(exp_measure_csv_path)

# concatenate rois and expanded measurements from all images
exp_measurements_df = concat_csvs(csv_dir/'expanded2_measurements')
rois_df = concat_csvs(csv_dir/'rois')
exp_merged_df = rois_df.merge(exp_measurements_df, on = ['label', 'image'])

exp_measurements_df.to_csv(csv_dir/'combined_expanded2_measurements.csv', index = False)
exp_merged_df.to_csv(csv_dir/'Exp042_raw_ploidy_expanded2.csv', index = False)

240129_balminF2_079 already expanded and measured. skipping
 processing 240129_balminF2_080
 processing 240129_balminF2_082
 processing 240129_balminF2_083
 processing 240129_balminF2_084
 processing 240129_balminF2_085
 processing 240129_balmin_009
 processing 240129_balmin_018
 processing 240129_balmin_019
 processing 240129_balmin_022
 processing 240129_balmin_041
 processing 240129_balmin_047
 processing 240129_balmin_053
 processing 240129_balmin_055
 processing 240129_balmin_059
 processing 240129_balmin_063
 processing 240129_balmin_064
 processing 240129_balmin_065
 processing 240129_balmin_066
 processing 240129_balmin_123
 processing 240129_balmin_223
 processing 240129_balplus_004
 processing 240129_balplus_011
 processing 240129_balplus_014
 processing 240129_balplus_015
 processing 240129_balplus_021
 processing 240129_balplus_024
 processing 240129_balplus_026
240129_balplus_031 does not have corrected labels. skipping
240129_balplus_034 does not have corrected labels. sk

In [3]:
from napari_simpleitk_image_processing import label_statistics
from skimage.segmentation import expand_labels

for image_path in image_path_list:

    # get image id and define file paths
    image_id = image_path.stem
    corr_label_path = data_dir/'corrected_labels'/f'{image_id}_corrected_labels.tif'
    exp_label_path = data_dir/'expanded1_corrected_labels'/f'{image_id}_expanded_corrected_labels.tif'
    exp_measure_csv_path = data_dir/'csvs'/'expanded1_measurements'/f'{image_id}_expanded_measurements.csv'

    # skip images without labels or that already have expanded labels
    if not corr_label_path.is_file():
        print(f'{image_id} does not have corrected labels. skipping')
        continue
    
    if exp_label_path.is_file():
        print(f'{image_id} already expanded and measured. skipping')
        continue
    
    print(f' processing {image_id}')
    
    # read image and labels
    im = imread(image_path)
    corr_labels = imread(corr_label_path)
    exp_labels = expand_labels(corr_labels, distance = 1)
    
    # save expanded labels and measurements
    imwrite(exp_label_path, exp_labels)
    measure_df = pd.DataFrame(label_statistics(im, exp_labels, position=True))
    measure_df['image'] = image_id
    measure_df.to_csv(exp_measure_csv_path)

# concatenate rois and expanded measurements from all images
exp_measurements_df = concat_csvs(csv_dir/'expanded1_measurements')
rois_df = concat_csvs(csv_dir/'rois')
exp_merged_df = rois_df.merge(exp_measurements_df, on = ['label', 'image'])

exp_measurements_df.to_csv(csv_dir/'combined_expanded1_measurements.csv', index = False)
exp_merged_df.to_csv(csv_dir/'Exp042_raw_ploidy_expanded1.csv', index = False)

 processing 240129_balminF2_079
 processing 240129_balminF2_080
 processing 240129_balminF2_082
 processing 240129_balminF2_083
 processing 240129_balminF2_084
 processing 240129_balminF2_085
 processing 240129_balmin_009
 processing 240129_balmin_018
 processing 240129_balmin_019
 processing 240129_balmin_022
 processing 240129_balmin_041
 processing 240129_balmin_047
 processing 240129_balmin_053
 processing 240129_balmin_055
 processing 240129_balmin_059
 processing 240129_balmin_063
 processing 240129_balmin_064
 processing 240129_balmin_065
 processing 240129_balmin_066
 processing 240129_balmin_123
 processing 240129_balmin_223
 processing 240129_balplus_004
 processing 240129_balplus_011
 processing 240129_balplus_014
 processing 240129_balplus_015
 processing 240129_balplus_021
 processing 240129_balplus_024
 processing 240129_balplus_026
240129_balplus_031 does not have corrected labels. skipping
240129_balplus_034 does not have corrected labels. skipping
240129_balplus_035 do

In [4]:
### check that expanded labels look reasonable
import napari
viewer = napari.Viewer()
viewer.add_image(im, contrast_limits = [0, 0.7 * int(np.max(im))])
viewer.add_labels(corr_labels, opacity = 0.4)
viewer.add_labels(exp_labels, opacity = 0.4)

Assistant skips harvesting pyclesperanto as it's not installed.


<Labels layer 'exp_labels' at 0x12cbba6d0>